In [2]:
import torch
import torch.nn as nn
from torch.nn import functional

device = 'cuda' if torch.cuda.is_available() else 'cpu'

block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250

In [3]:
with open("wizard_of_oz.txt", 'r', encoding = 'utf8') as f:
    text = f.read()

chars = sorted(set(text))
vocab_size = len(chars)

print(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [4]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_strings = {i:ch for i, ch in enumerate(chars)}

encode = lambda x: [string_to_int[c] for c in x]
decode = lambda n: ''.join([int_to_strings[l] for l in n])

data = torch.tensor(encode(text), dtype=torch.long)

print(data[:10])


tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49])


In [5]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    
    ix = torch.randint((len(data) - block_size), (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    x, y = x.to(device), y.to(device)
    
    return x, y

x, y = get_batch('train')

print('inputs: ')
print(x)
print(x.shape)

print('targets: ')
print(y)
print(y.shape)

inputs: 
tensor([[61,  1, 60, 65, 54, 72, 72,  1],
        [61, 58, 78,  1, 71, 58, 54, 56],
        [69,  0, 74, 72,  9,  3,  1, 72],
        [68, 59,  1, 73, 61, 58,  1, 37]])
torch.Size([4, 8])
targets: 
tensor([[ 1, 60, 65, 54, 72, 72,  1, 55],
        [58, 78,  1, 71, 58, 54, 56, 61],
        [ 0, 74, 72,  9,  3,  1, 72, 74],
        [59,  1, 73, 61, 58,  1, 37, 54]])
torch.Size([4, 8])


In [6]:
@torch.inference_mode()
def estimate_loss(model):
    out = {}
    model.eval()
    
    for split in ['train', 'test']:
        losses = torch.zeros(eval_iters)
        
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    
    model.train()
    return out
    

In [10]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = functional.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            
            logits , loss = self.forward(index)
            logits = logits[:,-1,:]
            
            probs = functional.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            
            index = torch.cat((index, index_next), dim=1)
            
        return index
    
model = BigramLanguageModel(vocab_size)
model = model.to(device)


context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(model.generate(context, 500)[0].tolist())
print(generated_chars)  


w[
(PgtoLme_﻿7pwWQ5t,
:&MK,
ZlFZ﻿)91ZWF2BK 1sx_OMK)zRbQEFunS6Pv?F28cu1eK:4Mwo(6(jm3Vf[A(S6V8K;uZkKZb.Z1riq?))z&P],??LMlK6Onyn2S7f)B [zGw)&o3cH;f3b5XT3sBBshD4v0pCJr4i,U_O
wSIj)rRGWE3rjbd0(N,pB i,st!oBK5uGWyJ-Jb.vvCGs,Q.ZV8-p
"EGLMx.CL*".3c.-IQf?KnAV*"MhBK5Hy[FxbcAn1iZkw,_h(9ZE:UDizB?﻿Jc*Ce1*x;﻿o:A Iqu.Pv?mJj39
nq?hFc*RaWAgm0*QfAVy1'qOn2NJ﻿y[ipjNK)3J0]qXYI4_8&rB&?c
:dc
([0d,AT'xTgt(X2H9y(TxYR0tP&&efM
WCUa)y"XE2gPu2mQ5 W

a)pNbAgj_
"'QqN1iQGXd RP .iUi,
It6?51Z*9Mjfna!.LXd;-Jt.izYD nJlH9JxHw﻿la-LWt!


In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(20000):
    if iter % eval_iters == 0:
        losses = estimate_loss(model)
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['test']:.3f}")
        
    x, y  = get_batch('train')
    logits, loss = model.forward(x, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()



step: 0, train loss: 4.133, val loss: 4.114
step: 250, train loss: 4.103, val loss: 4.083
step: 500, train loss: 4.013, val loss: 4.018
step: 750, train loss: 3.961, val loss: 3.981
step: 1000, train loss: 3.937, val loss: 3.915
step: 1250, train loss: 3.890, val loss: 3.882
step: 1500, train loss: 3.844, val loss: 3.856
step: 1750, train loss: 3.802, val loss: 3.796
step: 2000, train loss: 3.756, val loss: 3.741
step: 2250, train loss: 3.713, val loss: 3.700
step: 2500, train loss: 3.673, val loss: 3.666
step: 2750, train loss: 3.642, val loss: 3.644
step: 3000, train loss: 3.586, val loss: 3.605
step: 3250, train loss: 3.567, val loss: 3.565
step: 3500, train loss: 3.519, val loss: 3.528
step: 3750, train loss: 3.504, val loss: 3.496
step: 4000, train loss: 3.482, val loss: 3.451
step: 4250, train loss: 3.449, val loss: 3.447
step: 4500, train loss: 3.409, val loss: 3.389
step: 4750, train loss: 3.351, val loss: 3.361
step: 5000, train loss: 3.330, val loss: 3.335
step: 5250, train l